# Object Detection - YOLOS from Hugging Face

This model identifies objects present in images, returning its scores, labels and bounding boxes.


We are using a dataset from [UCF](https://www.crcv.ucf.edu/data/GMCP_Geolocalization/#Dataset) and the model [YOLOS](https://huggingface.co/hustvl/yolos-base) from [Hugging Face](https://huggingface.co/).

<a href="https://colab.research.google.com/github/VertaAI/examples/blob/" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

This notebook has been tested with **Python 3.8.15** and the following package versions:

In [ ]:
%%capture
!pip install verta==0.21.1
!pip install transformers==4.25.1

## Imports

In [ ]:
import os
import warnings

from transformers import pipeline, YolosForObjectDetection, YolosFeatureExtractor
from verta import Client
from verta.endpoint.autoscaling import Autoscaling
from verta.endpoint.autoscaling.metrics import CpuUtilizationTarget, MemoryUtilizationTarget, RequestsPerWorkerTarget
from verta.endpoint.resources import Resources
from verta.environment import Python
from verta.registry import VertaModelBase, verify_io
from verta.utils import ModelAPI

warnings.simplefilter(action='ignore', category=FutureWarning)

## Model Class

In [ ]:
class Yolos(VertaModelBase):
    def __init__(self, artifacts=None):
        self.weight = "tiny" # tiny/small/base
        self.model_name = f"hustvl/yolos-{self.weight}"
        self.model = pipeline(
            task = 'object-detection',
            model = YolosForObjectDetection.from_pretrained(self.model_name),
            feature_extractor = YolosFeatureExtractor.from_pretrained(self.model_name)
        )

    def example(self):
        return ['http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/000001_0.jpg']

    def describe(self):
        return {
            "method": "predict",
            "args": "[url]",
            "returns": "{'score': 0, 'label': '', 'box': {'xmin': 0, 'ymin': 0, 'xmax': 0, 'ymax': 0}}",
            "description": """
                Identifies objects present in the image list, returning its scores, labels and bounding boxes
            """,
            "input_description": """
                A list with image URLs.
            """,
            "output_description": """
                A JSON with each object score, label and bounding box.
            """
        }

    @verify_io
    def predict(self, urls):
        return self.model(urls)    

## Model Test

In [ ]:
yolos = Yolos()

In [ ]:
url = yolos.example()

In [ ]:
predictions = yolos.predict(url)

In [ ]:
url

In [ ]:
predictions

## Verta Set Up

In [ ]:
os.environ['VERTA_HOST'] = ''
os.environ['VERTA_EMAIL'] = ''
os.environ['VERTA_DEV_KEY'] = ''

## Model Registration 

In [ ]:
client = Client(os.environ['VERTA_HOST'])

In [ ]:
model_name = 'YOLOS'
model_version = 'v1'

In [ ]:
registered_model = client.get_or_create_registered_model(
    name = model_name,
    desc = 'Model for detecting objects in images.',
    labels = ['object-detection']
)

In [ ]:
model = registered_model.create_standard_model(
      name = model_version,
      model_cls = Yolos,
      model_api = ModelAPI([{'urls': url}], [{'predictions': predictions[0][0]}]),
      environment = Python(requirements = ['dill', 'Pillow', 'transformers', 'torch'])
)

## VM Auto Scaling and Resources Configuration

In [ ]:
autoscaling = Autoscaling(min_replicas=1, max_replicas=20, min_scale=0.1, max_scale=10)
autoscaling.add_metric(CpuUtilizationTarget(0.6))
autoscaling.add_metric(MemoryUtilizationTarget(0.7))
autoscaling.add_metric(RequestsPerWorkerTarget(1))
resources = Resources(cpu=2., memory='12Gi')

## Endpoint Creation

In [ ]:
endpoint_name = 'yolos'

In [ ]:
transformers_cache_env = {'TRANSFORMERS_CACHE': '/tmp'}

In [ ]:
endpoint = client.get_or_create_endpoint(endpoint_name)

In [ ]:
%%time
endpoint.update(
    model,
    autoscaling = autoscaling,
    resources = resources,
    env_vars = transformers_cache_env,
    wait = True
)

## Endpoint Test

In [ ]:
deployed_model = endpoint.get_deployed_model()

In [ ]:
url

In [ ]:
deployed_model.predict(url)